In [ ]:
#https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0/kagglepytorch%E3%81%A7digit-recognizer%E3%81%AB%E6%8C%91%E6%88%A6/
#https://www.kaggle.com/georgiisirotenko/pytorch-mnist-transferlearning-ensemble-99-714

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR="../input/digit-recognizer/"

# **Data Loader**

In [3]:
#前処理
transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.),std=(0.5,))
])

#データセット
class MyDataset(Dataset):
    def __init__(self,features,labels,Transform):
        self.x=features
        self.y=labels
        self.transform=Transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self,index):
        return self.transform(self.x[index]), self.y[index]

In [4]:
def GetDf(df,Transform):
    x_features=df.loc[:, df.columns !="label"].values/255
    y_labels =df.label.values
    x_features=x_features.reshape(-1,1,28,28)
    #x_features = np.uint8(x_features)
    
    x_features=torch.from_numpy(x_features)
    y_labels=torch.from_numpy(y_labels).type(torch.LongTensor)
    
    return MyDataset(x_features, y_labels,Transform)

In [5]:
#batch_size
batch_size=128


#Prepare Dataset
train_df=pd.read_csv(DATA_DIR+"train.csv", dtype=np.float32)
test_df=pd.read_csv(DATA_DIR+"test.csv", dtype=np.float32)


def create_dataloaders(df,test_size=0.2,batch_size=batch_size):
    train_data,valid_data=train_test_split(df,test_size=test_size)
    
    train_dataset=GetDf(train_data,Transform=transform)
    valid_dataset=GetDf(valid_data,Transform=transform)
    
    #Pytorch train and test sets
    train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader=torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader, valid_loader


train_loader, valid_loader= create_dataloaders(df=train_df,test_size=0.2,batch_size=batch_size)

# **Model**

In [6]:
# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(p=0.25)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(p=0.25)
        
        # Fully connected 1
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
     
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        out = self.dropout1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        out = self.dropout2(out)
        
        # flatten
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

In [7]:
if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"

In [8]:
#Create CNN
model=CNNModel().to(device)

#Cross Entropy Loss
error=nn.CrossEntropyLoss()

#SGD Optimizer
optimizer=torch.optim.SGD(model.parameters(), lr=0.01,weight_decay=1e-5)

In [9]:
#CNN model training
count=0
loss_list=[]
iteration_list=[]
accuracy_list=[]

#num_epochs=int(n_iters /(len(features_train)/batch_size))
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    for images,labels in train_loader:
        images, labels= images.to(device), labels.to(device)
        
        train=Variable(images) #images.view(-1,1,28,28)
        labels=Variable(labels)
        
        #Clear gradients
        optimizer.zero_grad()
        #Foward propagation
        outputs=model(train)
        #calculate softmax and loss entropy loss
        loss=error(outputs,labels)
        loss.backward()
        optimizer.step()
        
        count+=1
        
        #validation
        if count%50==0:
            correct=0
            total=0
            model.eval()
            for images, labels in valid_loader:
                images, labels= images.to(device), labels.to(device)
                
                valid=Variable(images.view(-1,1,28,28))
                labels=Variable(labels)
                
                outputs=model(valid)
                
                #Get predictions from the maximum value
                predicted=torch.max(outputs.data,1)[1]
                
                #Total number of labels
                total +=len(labels)
                
                correct +=(predicted==labels).sum()
            
            accuracy=100*correct/ float(total)
            
            #store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            
        if count%500==0:
            #Print loss
            print("Iteration: {} Loss: {} Accuracy: {} %".format(count, loss.data, accuracy))

In [10]:
# visualization loss 
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("CNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy 
plt.plot(iteration_list,accuracy_list,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("CNN: Accuracy vs Number of iteration")
plt.show()

# **submission**

In [11]:
class TestDataset(Dataset):
    def __init__(self,features,Transform):
        self.features=features.values.reshape(-1,28,28)/255
        self.targets=None
        self.transform=Transform
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, index):
        return self.transform(self.features[index])

In [12]:
test=TestDataset(test_df,Transform=transform)
test_loader = torch.utils.data.DataLoader(test,batch_size=batch_size, shuffle=False)

In [13]:
model.eval()
test_pred=torch.LongTensor().to(device)
for  data in test_loader:
    data= Variable(data).to(device=device, dtype=torch.float)
    output=model(data)
    pred=output.data.max(1, keepdim=True)[1]#torch.max(outputs, 1)
    test_pred=torch.cat((test_pred,pred),dim=0)

In [14]:
test_pred=test_pred.squeeze(1)
test_pred.shape

In [15]:
output = pd.DataFrame(
    {
        'ImageId':pd.Series(range(1,28001)),
        "Label":test_pred.to('cpu').detach().numpy().copy()
    }
)

output.to_csv('my_submission.csv',index=False)
print(output.head())

In [ ]:
#今回は使わなかったPrepare Dataset

train=pd.read_csv(DATA_DIR+"train.csv", dtype=np.float32)
features_numpy=train.loc[:, train.columns !="label"].values/255
targets_numpy=train.label.values

features_train, features_valid, targets_train, targets_valid=train_test_split(features_numpy, targets_numpy, test_size=0.2, random_state=42)

features_train=torch.from_numpy(features_train)
targets_train=torch.from_numpy(targets_train).type(torch.LongTensor)
features_valid = torch.from_numpy(features_valid)
targets_valid = torch.from_numpy(targets_valid).type(torch.LongTensor) 



#Pytorch train and valid sets
train=torch.utils.data.TensorDataset(features_train, targets_train)
valid=torch.utils.data.TensorDataset(features_valid, targets_valid)

#Pytorch train and valid sets
train_loader=torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
valid_loader=torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

#Pytorch test sets

test=pd.read_csv(DATA_DIR+"test.csv").values/255
features_test=torch.from_numpy(test)
test=torch.utils.data.TensorDataset(features_test)
test_loader = torch.utils.data.DataLoader(test,batch_size=batch_size, shuffle=False)